### Imports

In [2]:
import psycopg2 as ps
import logging
import pandas as pd
from folium.plugins import HeatMap
import branca.colormap as cm
import os
import folium
from dotenv import load_dotenv
from contextlib import contextmanager
import dotenv

### Env and logger

In [3]:
load_dotenv(override=True)

True

In [4]:
# Configure the logger
logger = logging.getLogger(__name__)  # Create a logger with the module's name
logger.setLevel(logging.INFO)         # Set the logging level (INFO, ERROR, DEBUG, etc.)

# Add a logging handler to output logs to the console
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)

# Create a logging format and attach it to the handler
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
console_handler.setFormatter(formatter)

### DB connection

In [5]:
@contextmanager
def get_db_conn(logger):
    cnx = None
    config = {
        "user": os.getenv("POSTGRES_USER"),
        "password": os.getenv("POSTGRES_PASSWORD"),
        "host": os.getenv("POSTGRES_HOST"),
        "dbname": os.getenv("POSTGRES_DB"),
    }
    # logger.info(config)
    try:
        cnx = ps.connect(**config)
        logger.info("connected to DB")
        yield cnx
    except Exception as err:
        logger.error(err)
        raise
    finally:
        if cnx:
            cnx.close()
            logger.info("DB connection closed")

In [6]:
# Function to execute a query and return results as a DataFrame
def run_query(query):
    with get_db_conn(logger) as conn:  # Using the connection context manager
        df = pd.read_sql_query(query, conn)  # Run the query and store the result in a DataFrame
        return df

### SQL queries

In [7]:
query_athens_for_sale = """--Step 1: selecting all the single listings with prefilters
SELECT p.*,
       NULL AS rn
FROM properties p
LEFT JOIN properties__group_properties_relations pgpr ON p.id = pgpr.property_id
WHERE pgpr.property_id IS NULL
  AND p.publication_date >= '2024-01-01'
  AND city='Athens'
  AND p.is_property_for_sale = 'true' -- Step 2: selecting one multilisting with the biggest number of photos for each property
UNION ALL
SELECT rp.*
FROM
  (SELECT p.*,
          ROW_NUMBER() OVER (PARTITION BY pgpr.group_property_id
                             ORDER BY p.num_photos DESC) AS rn
   FROM properties p
   JOIN properties__group_properties_relations pgpr ON p.id = pgpr.property_id
   WHERE p.publication_date >= '2024-01-01'
     AND city='Athens'
     AND p.is_property_for_sale = 'true' ) rp
WHERE rp.rn = 1"""

In [8]:
query_ltr_analytics = """
    WITH grouped_properties AS (
    SELECT 
        p.*, 
        pgpr.group_property_id,
        ROW_NUMBER() OVER (PARTITION BY pgpr.group_property_id ORDER BY p.id) AS rn
    FROM 
        properties p
    JOIN 
        properties__group_properties_relations pgpr 
        ON p.id = pgpr.property_id
)
SELECT 
    p.*, 
    NULL AS group_property_id,
    NULL AS rn 
FROM 
    task_google_spreadsheet tgs
JOIN 
    task_google_spreadsheet__properties_relations tgspr 
    ON tgs.id = tgspr.task_google_id
JOIN 
    properties p 
    ON tgspr.property_id = p.id
LEFT JOIN 
    properties__group_properties_relations pgpr 
    ON p.id = pgpr.property_id
WHERE 
    tgs.name IN ('#48_Athens_Apartments_R')
    AND pgpr.group_property_id IS NULL
    AND p.publication_date > '2024-01-01'
        
UNION

SELECT 
    gp.*
FROM 
    grouped_properties gp
JOIN 
    task_google_spreadsheet tgs 
    ON gp.group_property_id IS NOT NULL
JOIN 
    task_google_spreadsheet__properties_relations tgspr
    ON tgs.id = tgspr.task_google_id AND tgspr.property_id = gp.id
WHERE
    tgs.name IN ('#48_Athens_Apartments_R')
    AND gp.rn = 1
    AND gp.publication_date > '2024-01-01';"""

### Loading and filtering DFs

Loading LTR data from xe.gr in central Athens (published in 2024)

In [9]:
df_ltr_analytics = run_query(query_ltr_analytics)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


Filtering apartments with sqm below 20 and above 85, below 100 eur/month and above 5K eur/month rental price

In [10]:
df_ltr_analytics = df_ltr_analytics[
    (df_ltr_analytics['price'] <= 5000) &
    (df_ltr_analytics['price'] >= 100) &
    (df_ltr_analytics['square'] >= 20) &
    (df_ltr_analytics['square'] <= 85)]

Loading apartments-on-sale data from xe.gr in central Athens (published in 2024)

In [11]:
df_apartment_for_sale = run_query(query_athens_for_sale)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


Filtering properties with sqm below 20 and above 85, price below 50K and above 100K, price/sqm below 500 and above 10K

In [12]:
# Apply the filtering conditions
df_apartment_for_sale = df_apartment_for_sale[
    (df_apartment_for_sale['price'] <= 1000000) & 
    (df_apartment_for_sale['price'] >= 50000) &
    (df_apartment_for_sale['square'] >= 20) & 
    (df_apartment_for_sale['square'] <= 85) &
    (df_apartment_for_sale['price_sq_m'] >= 500) & 
    (df_apartment_for_sale['price_sq_m'] <= 10000)
]

Loading blueground data collected by PM

In [13]:
# Load the Excel file (assuming the file is named 'data.xlsx')
file_path = '../pmc_data/Blueground Listings.xlsx'
# Read the  into dataframes
df_blueground = pd.read_excel(file_path, sheet_name='list')  # Assuming the second sheet is named 'Sheet2'
# Convert the 'size_m2' column to numeric (invalid values become NaN)
df_blueground['sqm'] = pd.to_numeric(df_blueground['sqm'], errors='coerce')
df_blueground['bedrooms'] = pd.to_numeric(df_blueground['sqm'], errors='coerce')
# Sort the DataFrame by price in descending order
df_sorted_bg = df_blueground.sort_values(by='Price', ascending=False)

### Segmenting DFs by apartment type

In [14]:
def segment_apartments(df_sorted, bedrooms_col, sqm_col):
    
    # Check for studio apartments (1-bed apartments with sqm < 30)
    studio_1_bed = df_sorted[(df_sorted[bedrooms_col] == 1) & (df_sorted[sqm_col] < 30)]

    # Segmentation of 1-bedroom apartments
    small_1_bed = df_sorted[(df_sorted[bedrooms_col] == 1) & (df_sorted[sqm_col] >= 30) & (df_sorted[sqm_col] <= 45)]
    mid_1_bed = df_sorted[(df_sorted[bedrooms_col] == 1) & (df_sorted[sqm_col] > 45) & (df_sorted[sqm_col] <= 65)]
    large_1_bed = df_sorted[(df_sorted[bedrooms_col] == 1) & (df_sorted[sqm_col] > 65)]
    
    # Segmentation of 2-bedroom apartments
    small_2_bed = df_sorted[(df_sorted[bedrooms_col] == 2) & (df_sorted[sqm_col] >= 50) & (df_sorted[sqm_col] <= 70)]
    mid_2_bed = df_sorted[(df_sorted[bedrooms_col] == 2) & (df_sorted[sqm_col] > 70) & (df_sorted[sqm_col] <= 85)]

    # Store the results in a dictionary
    df_dict = {
        "Small 1-bed apartments": small_1_bed,
        "Mid 1-bed apartments": mid_1_bed,
        "Large 1-bed apartments": large_1_bed,
        "Small 2-bed apartments": small_2_bed,
        "Mid 2-bed apartments": mid_2_bed
    }
    
    # Add the studio category if there are any such rows
    if not studio_1_bed.empty:
        df_dict["Studio 1-bed apartments"] = studio_1_bed
    
    return df_dict

In [15]:
bg_dict = segment_apartments(df_sorted_bg, 'Bedrooms', 'sqm')
ltr_rental_dict = segment_apartments(df_ltr_analytics, 'bedrooms','square')
on_sale_dict = segment_apartments(df_apartment_for_sale, 'bedrooms','square')

### Creating heatmaps

In [16]:
def create_heatmap_with_layers(dataframes_dict, column_name, title, output_path):
    # Initialize the map centered around Athens
    m = folium.Map(location=[37.9838, 23.7275], zoom_start=13)

    # Create a base colormap that will be adjusted per layer
    base_colormap = cm.LinearColormap(
        colors=['blue', 'purple', 'yellow', 'red'],  # Colors for low, medium, and high prices
        caption='Rental Price (€)'  # Caption for the color scale
    )
    
    # Loop through each DataFrame and create a corresponding heatmap layer
    for category, dataframe in dataframes_dict.items():
        if not dataframe.empty:
            # Prepare heatmap data: [latitude, longitude, price]
            heat_data = [[row['latitude'], row['longitude'], row[column_name]] for index, row in dataframe.iterrows()]
            
            # Calculate min and max prices for the current layer
            min_price = dataframe[column_name].min()
            max_price = dataframe[column_name].max()

            # Create a colormap for the current layer based on its min and max prices
            layer_colormap = cm.LinearColormap(
                colors=['blue', 'purple', 'yellow', 'red'],
                vmin=min_price,
                vmax=max_price,
                caption=f'{column_name} for {category}'  # Caption for the color scale
            )
            layer_colormap.add_to(m)

            # Create a FeatureGroup for this category
            feature_group = folium.FeatureGroup(name=category, control=True)
            
            # Create the heatmap layer
            HeatMap(heat_data, 
                    radius=15, 
                    blur=10, 
                    max_zoom=1, 
                    min_opacity=0.2, 
                    gradient={0: 'blue', 0.33: 'purple', 0.66: 'yellow', 1: 'red'},  # Color gradient
                    ).add_to(feature_group)
                    
            # Add the feature group to the map
            feature_group.add_to(m)

    # Add LayerControl to toggle between different heatmap layers
    folium.LayerControl(collapsed=False).add_to(m)

    # Save the map
    m.save(output_path)
    return m

In [17]:
# Create a directory for heatmaps if it doesn't exist
output_dir = '../heat_maps' 
os.makedirs(output_dir, exist_ok=True)

In [22]:
on_sale_heatmap_with_layers = create_heatmap_with_layers(on_sale_dict, "price_sq_m", "Xe.gr Sale Prices Heatmap with Layers", os.path.join(output_dir, 'on-sale_prices_per_sqm_with_layers.html'))

ltr_rental_heatmap_with_layers = create_heatmap_with_layers(ltr_rental_dict, "price", "Xe.gr Rental Prices Heatmap with Layers", os.path.join(output_dir, 'xe.gr_rental_prices_with_layers.html'))

blueground_rental_heatmap_with_layers = create_heatmap_with_layers(bg_dict, "Price", "Blueground Rental Prices Heatmap with Layers", os.path.join(output_dir, 'blueground_rental_prices_with_layers.html'))


In [19]:
on_sale_heatmap_with_layers

In [20]:
ltr_rental_heatmap_with_layers

In [21]:
blueground_rental_heatmap_with_layers